## Installing Tabgan

In [1]:
CMD = "wget https://raw.githubusercontent.com/Diyago/"\
  "GAN-for-tabular-data/master/requirements.txt"

!{CMD}
!pip install -r requirements.txt
!pip install tabgan

--2023-07-31 22:14:07--  https://raw.githubusercontent.com/Diyago/GAN-for-tabular-data/master/requirements.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 183 [text/plain]
Saving to: ‘requirements.txt.1’

requirements.txt.1  100%[===================>]     183  --.-KB/s    in 0s      

2023-07-31 22:14:07 (9.64 MB/s) - ‘requirements.txt.1’ saved [183/183]



---
## Loading the Libraries and the Auto MPG Dataset


In [90]:
from sklearn.model_selection import train_test_split
import pandas as pd
import io
import os
import requests
import numpy as np
from sklearn import metrics

In [91]:
from google.colab import files
uploaded = files.upload()


Saving auto-mpg.csv to auto-mpg (1).csv


In [103]:
# Load dataset and fill ? in dataset to NA
df = pd.read_csv("auto-mpg.csv", na_values=['NA', '?'])

In [104]:
# Checking for missing values
print(df.isna().sum().sum())

6


In [105]:
df.dropna(inplace=True)                       # Dropping all na values
df.drop('car name', axis=1, inplace=True)     # Dropping the car name column

In [106]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,18.0,8,307.0,130.0,3504,12.0,70,1
1,15.0,8,350.0,165.0,3693,11.5,70,1
2,18.0,8,318.0,150.0,3436,11.0,70,1
3,16.0,8,304.0,150.0,3433,12.0,70,1
4,17.0,8,302.0,140.0,3449,10.5,70,1
...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86.0,2790,15.6,82,1
394,44.0,4,97.0,52.0,2130,24.6,82,2
395,32.0,4,135.0,84.0,2295,11.6,82,1
396,28.0,4,120.0,79.0,2625,18.6,82,1


In [107]:
# Splitting into training and testing sets
df_x_train, df_x_test, df_y_train, df_y_test = train_test_split(
    df.drop("mpg", axis=1),   #X val
    df["mpg"],                #Y val
    test_size=0.20,           #Train: 80%, Test: 20%
    random_state=42,
)

In [109]:
# Creating new indexes by scrapping the old index
df_x_test = df_x_test.reset_index(drop=True)
df_y_test = df_y_test.reset_index(drop=True)

# Creating dataframe versions for tabular GAN
df_y_train = pd.DataFrame(df_y_train)
df_y_test = pd.DataFrame(df_y_test)

In [110]:
# Pandas to Numpy to feed to the Neural Network
x_train = df_x_train.values
x_test = df_x_test.values
y_train = df_y_train.values
y_test = df_y_test.values

---
## Creating an ANN Model and training it on the non-augmented Auto-MPG dataset

In [139]:
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import mean_squared_error

In [112]:
# Building the ANN
model = Sequential()

# Adding the Layers
model.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))  # Hidden Layer 1
model.add(Dense(64, activation='relu'))     # Hidden Layer 2
model.add(Dense(32, activation='relu'))     # Hidden Layer 3
model.add(Dense(16, activation='relu'))     # Hidden Layer 4
model.add(Dense(8, activation='relu'))      # Hidden Layer 5
model.add(Dense(1))                         # Output Layer

# Compiling the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the data
model.fit(x_train,y_train,validation_data=(x_test,y_test), epochs=100)

Epoch 1/100
10/10 [==============================] - 1s 21ms/step - loss: 2043.6477 - val_loss: 633.0054
Epoch 2/100
10/10 [==============================] - 0s 7ms/step - loss: 339.4521 - val_loss: 236.3034
Epoch 3/100
10/10 [==============================] - 0s 8ms/step - loss: 222.3315 - val_loss: 190.1904
Epoch 4/100
10/10 [==============================] - 0s 5ms/step - loss: 199.5111 - val_loss: 172.0918
Epoch 5/100
10/10 [==============================] - 0s 6ms/step - loss: 195.9111 - val_loss: 160.3929
Epoch 6/100
10/10 [==============================] - 0s 5ms/step - loss: 175.0292 - val_loss: 155.5907
Epoch 7/100
10/10 [==============================] - 0s 6ms/step - loss: 172.3656 - val_loss: 154.1425
Epoch 8/100
10/10 [==============================] - 0s 5ms/step - loss: 165.2777 - val_loss: 142.9912
Epoch 9/100
10/10 [==============================] - 0s 8ms/step - loss: 154.8110 - val_loss: 133.4980
Epoch 10/100
10/10 [==============================] - 0s 7ms/step - los


## RMSE of Non-Augmented dataset

In [113]:
# Use the trained model to predict values from 'x_test'
y_pred_basic_df = model.predict(x_test)

# Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_pred_basic_df, y_test))
print(f"Root Mean Squared Error (MSE): {rmse}")

3/3 [==============================] - 0s 4ms/step
Root Mean Squared Error (MSE): 3.144905301085799


---
## Creating Synthetic Tabular Data using GAN

In [124]:
from tabgan.sampler import GANGenerator
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

synthetic_x, synthetic_y = GANGenerator(
    gen_x_times=0.05,                # 60% of the total number of data will be generated
    cat_cols=None,                  # No categorical columns
    bot_filter_quantile=0.001,      # Filtering out outliers under .1%-th quantile
    top_filter_quantile=0.999,      # Filtering out outliers over 99.9%-th quantile
    is_post_process=True,           # Whether post processing is going to be
                                    # performed on generated data

    # Training the adversarial model
    adversarial_model_params={
        "metrics": "rmse",        # Evaluation Metric of the adv model
        "max_depth": 2,           # Depth of the regression tree in grad boosting model
        "max_bin": 100,           # Maximum number of bins to discretize the continuous features
        "learning_rate": 0.02,    # Learning rate
        "random_state": 42,       # Random State
        "n_estimators": 500,      # Number of Regression Trees in the Grad Boosting ensemble
    },
    pregeneration_frac=1,         # Fraction of total data used to train adv mdoel
    only_generated_data=False,    # Only generated data not used
    gan_params={"batch_size": 50, "patience": 25, "epochs": 100} # Patience is number of eopchs until which it gives up (Early Stopping)
).generate_data_pipe(
    df_x_train,
    df_y_train,
    df_x_test,
    deep_copy=True,
    only_adversarial=False,
    use_adversarial=True
)



Fitting CTGAN transformers for each column:   0%|          | 0/8 [00:00<?, ?it/s]

Training CTGAN, epochs::   0%|          | 0/100 [00:00<?, ?it/s]

In [125]:
synthetic_x

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,4,151.0,90.0,3003,20.1,80,1
1,6,225.0,90.0,3381,18.7,80,1
2,6,231.0,105.0,3535,19.2,78,1
3,6,250.0,98.0,3525,19.0,77,1
4,6,225.0,110.0,3620,18.7,78,1
...,...,...,...,...,...,...,...
287,4,97.0,88.0,2130,14.5,71,3
288,4,88.0,76.0,2065,14.5,71,2
289,4,97.0,92.0,2288,17.0,72,3
290,4,97.0,88.0,2100,16.5,72,3


In [126]:
synthetic_y

0      24.3
1      19.1
2      19.2
3      18.5
4      18.6
       ... 
287    27.0
288    30.0
289    28.0
290    27.0
291    24.0
Name: mpg, Length: 292, dtype: float64

In [138]:
# Converting the synthetic y to a dataframe to append it to the original training dataset
synthetic_y = pd.DataFrame(synthetic_y)

In [129]:
df_x_train

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
260,6,225.0,110.0,3620,18.7,78,1
184,4,140.0,92.0,2572,14.9,76,1
174,6,171.0,97.0,2984,14.5,75,1
64,8,318.0,150.0,4135,13.5,72,1
344,4,86.0,64.0,1875,16.4,81,1
...,...,...,...,...,...,...,...
72,8,304.0,150.0,3892,12.5,72,1
107,6,232.0,100.0,2789,15.0,73,1
272,4,151.0,85.0,2855,17.6,78,1
352,4,98.0,65.0,2380,20.7,81,1


In [132]:
df_y_train

,mpg
260,18.6
184,25.0
174,18.0
64,15.0
344,39.0
...,...
72,15.0
107,18.0
272,23.8
352,29.9


---
## Appending the augmented data to the training data

In [133]:
augmented_x = df_x_train.append(synthetic_x, ignore_index=True)
augmented_x

,cylinders,displacement,horsepower,weight,acceleration,model year,origin
0,6,225.0,110.0,3620,18.7,78,1
1,4,140.0,92.0,2572,14.9,76,1
2,6,171.0,97.0,2984,14.5,75,1
3,8,318.0,150.0,4135,13.5,72,1
4,4,86.0,64.0,1875,16.4,81,1
...,...,...,...,...,...,...,...
600,4,97.0,88.0,2130,14.5,71,3
601,4,88.0,76.0,2065,14.5,71,2
602,4,97.0,92.0,2288,17.0,72,3
603,4,97.0,88.0,2100,16.5,72,3


In [134]:
augmented_y = df_y_train.append(synthetic_y, ignore_index=True)
augmented_y

,mpg
0,18.6
1,25.0
2,18.0
3,15.0
4,39.0
...,...
600,27.0
601,30.0
602,28.0
603,27.0


---
## Creating the same ANN Model and training it on the Augmented Auto-MPG dataset

In [135]:
# Pandas to Numpy to feed to the Neural Network
aug_x_train = augmented_x.values
aug_y_train = augmented_y.values

In [136]:
# Building the ANN
aug_model = Sequential()

# Adding the Layers
aug_model.add(Dense(128, input_dim=x_train.shape[1], activation='relu'))  # Hidden Layer 1
aug_model.add(Dense(64, activation='relu'))     # Hidden Layer 2
aug_model.add(Dense(32, activation='relu'))     # Hidden Layer 3
aug_model.add(Dense(16, activation='relu'))     # Hidden Layer 4
aug_model.add(Dense(8, activation='relu'))      # Hidden Layer 5
aug_model.add(Dense(1))                         # Output Layer

# Compiling the model
aug_model.compile(optimizer='adam', loss='mean_squared_error')

# Fitting the data
aug_model.fit(aug_x_train, aug_y_train,validation_data=(x_test,y_test), epochs=100)

Epoch 1/100
19/19 [==============================] - 1s 13ms/step - loss: 258.3829 - val_loss: 163.3542
Epoch 2/100
19/19 [==============================] - 0s 5ms/step - loss: 134.5571 - val_loss: 104.7187
Epoch 3/100
19/19 [==============================] - 0s 4ms/step - loss: 109.0690 - val_loss: 107.7929
Epoch 4/100
19/19 [==============================] - 0s 6ms/step - loss: 110.8134 - val_loss: 63.7941
Epoch 5/100
19/19 [==============================] - 0s 4ms/step - loss: 86.7905 - val_loss: 72.9363
Epoch 6/100
19/19 [==============================] - 0s 5ms/step - loss: 70.7955 - val_loss: 57.1872
Epoch 7/100
19/19 [==============================] - 0s 4ms/step - loss: 58.7097 - val_loss: 37.1755
Epoch 8/100
19/19 [==============================] - 0s 5ms/step - loss: 49.4854 - val_loss: 37.0683
Epoch 9/100
19/19 [==============================] - 0s 4ms/step - loss: 43.3729 - val_loss: 33.2197
Epoch 10/100
19/19 [==============================] - 0s 5ms/step - loss: 39.9881 -

## RMSE of the Augmented dataset

In [137]:
# Use the trained augmented model to predict values from 'x_test'
y_pred_aug_df = aug_model.predict(x_test)

# Root Mean Squared Error
rmse = np.sqrt(mean_squared_error(y_pred_aug_df, y_test))
print(f"Root Mean Squared Error (MSE): {rmse}")

3/3 [==============================] - 0s 4ms/step
Root Mean Squared Error (MSE): 2.97548003824662
